# -----Importing the libraries ----#

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import numpy as np
from matplotlib import style
import datetime as dt
from functools import reduce 
import time
import pandas as pd
from IPython.display import clear_output
import re
import ta
from scipy.signal import argrelextrema
from scipy.signal import find_peaks, peak_prominences
import seaborn as sns
%matplotlib inline
from sklearn import preprocessing
from scipy.spatial.distance import euclidean
# from fastdtw import fastdtw
from ta import *
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import calendar
from dateutil import relativedelta
import warnings
warnings.filterwarnings("ignore")
import itertools


from datetime import date, timedelta as td

from functools import reduce
import inspect
from functools import partial
from multiprocessing import Pool
# import talib

# ------ Rules ----------- #
Momentum_s1

1. When the close crosses above the 100-day high of the close, we go long at the close.
2. When the close crosses below the lowest close in the last 100 days, we sell at the close.

In [2]:
strategy_name = 'Momentum_s1' # ----------- Edit the Strategy_name ------------- #

In [3]:
# ---------- Paths -----------#

#-------------- export path -----------#

initial_export_path = "E:/WSG Markets/Backtesting V2 Nov 2025/BT results/" + strategy_name + "/"

export_path_Open_Trades =  initial_export_path + "/Open_Trades/"
export_path_trade_level_path =  initial_export_path + "/Trade Level Data/"
export_path_BT =  initial_export_path + "/BT/"
export_path_FT =  initial_export_path + "/FT/"
path_for_error_files = initial_export_path + "/Error/"

try:
    os.mkdir(initial_export_path)
    os.mkdir(export_path_Open_Trades)
    os.mkdir(export_path_trade_level_path)
    os.mkdir(export_path_BT)
    os.mkdir(export_path_FT)
    os.mkdir(path_for_error_files)

except:
    print("Folder Already Created")

# #-------------- Entry data export path -----------#
# Entry_data_export_path = "E:/WSG Markets/MA-RSI backtesting/Exit by RSI value/Exit Data/"

#-------------- fxcm data export path -----------#
fxcm_data_export_path = "E:/WSG Markets/Backtesting V2 Nov 2025/Raw Data/"

Folder Already Created


# -----Loading the data -----#

In [4]:


#----------d1 Data -----------#
d1 = pd.read_csv(fxcm_data_export_path + "Master_Data till 31 0ct 2025.csv") # ------ it is Fwd looking data ------- #
d1['date'] = pd.to_datetime(d1['date'],format = '%d-%m-%Y')
d1.rename(columns = {'Symbol':'Currency'},inplace=True)
d1['Timeframe'] = "D1"

price_data = pd.concat([d1],axis=0,ignore_index=True)


price_data.tail()

,date,Currency,org_symbol,askopen,askhigh,asklow,askclose,bidopen,bidhigh,bidlow,bidclose,volume,Timeframe
532849,2025-10-29,ZIO/USD,ZION,52.06216,52.52805,51.21796,51.32008,52.06216,52.52805,51.21796,51.32008,61405,D1
532850,2025-10-30,ZIO/USD,ZION,52.65004,53.00907,52.09756,52.17012,52.65004,53.00907,52.09756,52.17012,29674,D1
532851,2025-10-31,ZIO/USD,ZION,51.96395,52.23742,51.40534,52.07465,51.96395,52.23742,51.40534,52.07465,36315,D1
532852,2025-03-11,ZIO/USD,ZION,51.64933,51.97915,51.46341,51.79528,51.64933,51.97915,51.46341,51.79528,2637900,D1
532853,2025-04-11,ZIO/USD,ZION,51.75178,51.88266,51.28673,51.45338,51.75178,51.88266,51.28673,51.45338,3270400,D1


# ---------------------- Combinations Iteration List ----------------------- #

In [5]:
strategy = [strategy_name]


symbols = price_data['Currency'].unique()

symbols_not_to_BT = ["XTI/USD","XNG/USD","USD/TRY","EUR/TRY","USD/CNH","TRY/JPY"]

symbols = list(set(symbols) - set(symbols_not_to_BT))


condition = ["Long"]

tf = ["D1"]

# -------- Choose the parameter as per strategy -------------- #

parameter_1 = list(range(50,210,10)) # n days
parameter_2 = list(range(10,11,1)) # parameter not required, keeping it constant.
parameter_3 = list(range(10,11,1)) # parameter not required, keeping it constant.
parameter_4 = list(range(10,11,1)) # parameter not required, keeping it constant.
parameter_5 = list(range(10,11,1)) # parameter not required, keeping it constant.
parameter_6 = list(range(10,11,1)) # parameter not required, keeping it constant.


all_combinations = itertools.product(strategy,symbols, condition,tf,
                                     parameter_1,parameter_2,parameter_3,parameter_4,parameter_5,parameter_6)


iter_list = pd.DataFrame(all_combinations, columns=['Strategy','Currency','Condition','Timeframe',
                                                    'parameter_1','parameter_2','parameter_3','parameter_4',
                                                    'parameter_5','parameter_6'])



iter_list['Quote_Curr'] = iter_list['Currency'].str[4:]

In [6]:
iter_list

,Strategy,Currency,Condition,Timeframe,parameter_1,parameter_2,parameter_3,parameter_4,parameter_5,parameter_6,Quote_Curr
0,Momentum_s1,HON/USD,Long,D1,50,10,10,10,10,10,USD
1,Momentum_s1,HON/USD,Long,D1,60,10,10,10,10,10,USD
2,Momentum_s1,HON/USD,Long,D1,70,10,10,10,10,10,USD
3,Momentum_s1,HON/USD,Long,D1,80,10,10,10,10,10,USD
4,Momentum_s1,HON/USD,Long,D1,90,10,10,10,10,10,USD
...,...,...,...,...,...,...,...,...,...,...,...
3371,Momentum_s1,QCO/USD,Long,D1,160,10,10,10,10,10,USD
3372,Momentum_s1,QCO/USD,Long,D1,170,10,10,10,10,10,USD
3373,Momentum_s1,QCO/USD,Long,D1,180,10,10,10,10,10,USD
3374,Momentum_s1,QCO/USD,Long,D1,190,10,10,10,10,10,USD


In [7]:
iteration_list_1 = iter_list.values.tolist()
# iteration_list_1 = iteration_list_1[:1]
iteration_list_1

[['Momentum_s1', 'HON/USD', 'Long', 'D1', 50, 10, 10, 10, 10, 10, 'USD'],
 ['Momentum_s1', 'HON/USD', 'Long', 'D1', 60, 10, 10, 10, 10, 10, 'USD'],
 ['Momentum_s1', 'HON/USD', 'Long', 'D1', 70, 10, 10, 10, 10, 10, 'USD'],
 ['Momentum_s1', 'HON/USD', 'Long', 'D1', 80, 10, 10, 10, 10, 10, 'USD'],
 ['Momentum_s1', 'HON/USD', 'Long', 'D1', 90, 10, 10, 10, 10, 10, 'USD'],
 ['Momentum_s1', 'HON/USD', 'Long', 'D1', 100, 10, 10, 10, 10, 10, 'USD'],
 ['Momentum_s1', 'HON/USD', 'Long', 'D1', 110, 10, 10, 10, 10, 10, 'USD'],
 ['Momentum_s1', 'HON/USD', 'Long', 'D1', 120, 10, 10, 10, 10, 10, 'USD'],
 ['Momentum_s1', 'HON/USD', 'Long', 'D1', 130, 10, 10, 10, 10, 10, 'USD'],
 ['Momentum_s1', 'HON/USD', 'Long', 'D1', 140, 10, 10, 10, 10, 10, 'USD'],
 ['Momentum_s1', 'HON/USD', 'Long', 'D1', 150, 10, 10, 10, 10, 10, 'USD'],
 ['Momentum_s1', 'HON/USD', 'Long', 'D1', 160, 10, 10, 10, 10, 10, 'USD'],
 ['Momentum_s1', 'HON/USD', 'Long', 'D1', 170, 10, 10, 10, 10, 10, 'USD'],
 ['Momentum_s1', 'HON/USD', 'L

In [8]:
# completed_file_path = export_path_trade_level_path
# completed_file = os.listdir(completed_file_path)
# completed_file



completed_file_path = export_path_trade_level_path

error_completed_file_path = path_for_error_files

completed_file_1 = os.listdir(completed_file_path)

completed_file_2 = os.listdir(error_completed_file_path)

completed_file_2 = [word.replace('_error', "") for word in completed_file_2]

completed_file = completed_file_1 + completed_file_2

print(len(completed_file_1),len(completed_file_2),len(completed_file))

completed_file

0 6752 6752


['ACN_USD_D1_100_10_10_10_10_10_Long.parquet',
 'ACN_USD_D1_100_10_10_10_10_10_Short.parquet',
 'ACN_USD_D1_110_10_10_10_10_10_Long.parquet',
 'ACN_USD_D1_110_10_10_10_10_10_Short.parquet',
 'ACN_USD_D1_120_10_10_10_10_10_Long.parquet',
 'ACN_USD_D1_120_10_10_10_10_10_Short.parquet',
 'ACN_USD_D1_130_10_10_10_10_10_Long.parquet',
 'ACN_USD_D1_130_10_10_10_10_10_Short.parquet',
 'ACN_USD_D1_140_10_10_10_10_10_Long.parquet',
 'ACN_USD_D1_140_10_10_10_10_10_Short.parquet',
 'ACN_USD_D1_150_10_10_10_10_10_Long.parquet',
 'ACN_USD_D1_150_10_10_10_10_10_Short.parquet',
 'ACN_USD_D1_160_10_10_10_10_10_Long.parquet',
 'ACN_USD_D1_160_10_10_10_10_10_Short.parquet',
 'ACN_USD_D1_170_10_10_10_10_10_Long.parquet',
 'ACN_USD_D1_170_10_10_10_10_10_Short.parquet',
 'ACN_USD_D1_180_10_10_10_10_10_Long.parquet',
 'ACN_USD_D1_180_10_10_10_10_10_Short.parquet',
 'ACN_USD_D1_190_10_10_10_10_10_Long.parquet',
 'ACN_USD_D1_190_10_10_10_10_10_Short.parquet',
 'ACN_USD_D1_200_10_10_10_10_10_Long.parquet',
 'A

In [9]:
iteration_list = []
for file_item in iteration_list_1:
        
    file_name = (file_item[1].replace("/","_") + "_" + file_item[3] + "_" + str(file_item[4]) + "_" + str(file_item[5]) + "_" +
                    str(file_item[6]) + "_" + str(file_item[7]) + "_" + 
                     str(file_item[8]) + "_" + str(file_item[9]) + "_" + file_item[2] + ".parquet")
    
    
    if file_name in completed_file:
        a = "completed"
#         print(file_name)
#         print("Completed")
    else:
        iteration_list.append(file_item)
    
#     break

In [10]:
len(iteration_list)

0

# ----Main Algo -----#

In [11]:
def Func(it,iteration_list,price_data,initial_export_path): 
    
    
    import numpy as np
    import ta
    from ta import trend
    from ta.trend import sma_indicator
    import pandas as pd
    import numpy as np
    import pandas
    import numpy
    import ta
    from tqdm.notebook import tqdm
    from time import time
    from datetime import date, timedelta as td
    import datetime as dt
    from multiprocessing import Pool
    from multiprocessing import cpu_count
    from IPython.display import display
    import os
    from multiprocessing import Pool
    from functools import partial
    import inspect
    from IPython.display import clear_output
#     import talib
    
    import pyarrow.parquet as pq
    import pyarrow
    from pathlib import Path
    
    
    
    i = iteration_list[it]
    Strategy = i[0]
    currency = i[1]
    condition = i[2]
    timeframe = i[3]
    sym1 = currency.replace('/','_')

    parameter_1 = int(i[4])
    parameter_2 = int(i[5])
    parameter_3 = int(i[6])
    parameter_4 = int(i[7])
    parameter_5 = int(i[8])
    parameter_6 = int(i[9])

    #-----------------------------------------#
    
    sm = currency.find('JPY') 
    if sm != -1:
        val = (1/100) 
    else:
        val = (1/10000)
        
        
    sym = currency[4:]
    sym1= currency[0:3]+"_"+currency[4:]
        
    #----------------------------------------#
    
    #-----Filtering the raw data ------#
    df = price_data[(price_data['Timeframe'] == timeframe)&\
                    (price_data['Currency'] == currency)].reset_index(drop = True)
    
    # -------------- Current_Price ----------------------- #
    current_price_data = df.tail(1).reset_index(drop = True)
    
    #--------------------------------------------#
    
    
    df['parameter_1'] = parameter_1
    df['parameter_2'] = parameter_2
    df['parameter_3'] = parameter_3
    df['Strategy'] = Strategy
    df['parameter_4'] = parameter_4
    df['parameter_5'] = parameter_5
    df['parameter_6'] = parameter_6
    
    
    # ---------- Write the Entry signal as per strategy ----------------- #
    
    # ------------- Entry Signal ----------------------#
    
    
    
    df['nd_High'] = df['askclose'].rolling(window=parameter_1).max()

    df['Entry_signal'] = np.where((df['askclose'] > df['nd_High']) &
                                  (df['askclose'].shift(1) <= df['nd_High'].shift(1)),"Long", 0)

    df['Condition'] = df['Entry_signal']

    df['Trading Price Point'] = np.where((df['Entry_signal'] == "Long"),df['askclose'],0)

    df['TRADE_OPEN_Flag'] = np.where(df['Entry_signal'] == "Long",1,0)

    
    
    # ------------ Write the Exit signal as per startegy -------------- #

    df['Exit_condition_1'] = np.where(df['askclose'] > df['askhigh'].shift(1),1,0)

    df['Exit_condition_2'] = np.where(df['TRADE_OPEN_Flag'].shift(3) == 1,1,0)

    df['Exit_condition'] = (df['Exit_condition_1'] + df['Exit_condition_2'])

    df['Exit_Signal'] = np.where(df['Exit_condition'] >= 1,1,0)
    
    
    # --------------- Filtering Open Trades ---------------------- #
    
    df['Trade_Status_Flag'] = np.where(df['TRADE_OPEN_Flag'] == 1,"Open_First","Close")
    
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date').reset_index(drop = True)
    
    df = df.reset_index()
    open_data = df[df['Trade_Status_Flag'] == 'Open_First']
    open_data['date'] = pd.to_datetime(open_data['date'])
    open_data = open_data.reset_index(drop = True)

    # ------------------- Finding RSI value for exiting the trade ---------------- 3

    
    for i in range(0,open_data.shape[0]):

        subset_data = df[(df['index'] > open_data.loc[i,'index'])
                        & (df['Exit_Signal'] == 1)]

        try:
            ###select first row from the list#####
            min_value = min(subset_data['index'])

            RSI_subset_data = subset_data[subset_data['index'] == min_value]
            RSI_subset_data = RSI_subset_data.reset_index(drop = True)
            
            df.loc[open_data.loc[i,'index'],'Exit_askclose'] = RSI_subset_data['askclose'][0]
            df.loc[open_data.loc[i,'index'],'Exit_askopen'] = RSI_subset_data['askopen'][0]
            df.loc[open_data.loc[i,'index'],'Exit_asklow'] = RSI_subset_data['asklow'][0]
            df.loc[open_data.loc[i,'index'],'Exit_askhigh'] = RSI_subset_data['askhigh'][0]
            df.loc[open_data.loc[i,'index'],'Exit_bidclose'] = RSI_subset_data['bidclose'][0]
            df.loc[open_data.loc[i,'index'],'Exit_bidopen'] = RSI_subset_data['bidopen'][0]
            df.loc[open_data.loc[i,'index'],'Exit_bidlow'] = RSI_subset_data['bidlow'][0]
            df.loc[open_data.loc[i,'index'],'Exit_bidhigh'] = RSI_subset_data['bidhigh'][0]
            df.loc[open_data.loc[i,'index'],'Closing Date'] = RSI_subset_data['date'][0]
            df.loc[open_data.loc[i,'index'],'RSI_exit_status'] = "RSI_exit_reached"
            
        except:
            
            df.loc[open_data.loc[i,'index'],'Exit_askclose'] = current_price_data['askclose'][0]
            df.loc[open_data.loc[i,'index'],'Exit_askopen'] = current_price_data['askopen'][0]
            df.loc[open_data.loc[i,'index'],'Exit_asklow'] = current_price_data['asklow'][0]
            df.loc[open_data.loc[i,'index'],'Exit_askhigh'] = current_price_data['askhigh'][0]
            df.loc[open_data.loc[i,'index'],'Exit_bidclose'] = current_price_data['bidclose'][0]
            df.loc[open_data.loc[i,'index'],'Exit_bidopen'] = current_price_data['bidopen'][0]
            df.loc[open_data.loc[i,'index'],'Exit_bidlow'] = current_price_data['bidlow'][0]
            df.loc[open_data.loc[i,'index'],'Exit_bidhigh'] = current_price_data['bidhigh'][0]
            df.loc[open_data.loc[i,'index'],'Closing Date'] = current_price_data['date'][0]
            df.loc[open_data.loc[i,'index'],'RSI_exit_status'] = "RSI_exit_not_reached"
            
    # -------------------------------------------------------------------------------------------- #
    
    
    
            
    try:
        
        # ----------------------- Open trades ---------------------------------- #
        unclosed_trades = df[df['RSI_exit_status'] == 'RSI_exit_not_reached'].reset_index(drop = True)
        print(len(unclosed_trades))

        unclosed_trades['Total Amount'] = 100000
        unclosed_trades['Risk_Percent'] = 0.01

        unclosed_trades['Units'] = (unclosed_trades['Total Amount']/unclosed_trades['Trading Price Point']).round()

        # ----------------------------------------------------------------------- #



        # -------------- Filtering Open Trades and calculating profits---------------------------- #
        open_trades = df[df['RSI_exit_status'] == 'RSI_exit_reached']

        open_trades['Total Amount'] = 100000
        open_trades['Risk_Percent'] = 0.01

    #         open_trades['Units'] = ((open_trades['Risk_Percent']*open_trades['Total Amount'])/(abs(open_trades['Trading Price Point'] - open_trades['Stop_Loss']))).round()

        open_trades['Units'] = (open_trades['Total Amount']/open_trades['Trading Price Point']).round()

        open_trades['Profit_Loss'] = np.where((open_trades['Entry_signal'] == 'Long'),
                                ((open_trades['Units']*open_trades["Exit_bidclose"])-(open_trades['Units']*open_trades['Trading Price Point'])),
                            ((open_trades['Units']*open_trades['Trading Price Point'])-(open_trades['Units']*open_trades['Exit_askclose'])))

        open_trades['Good_Bad_Flag'] = np.where(open_trades['Profit_Loss']>0,1,0)


        # ----------------------------------------------------------------------------------------- #


    #     open_trades['Profit_Loss'] = open_trades['Profit_Loss']/20000
        open_trades['date'] = pd.to_datetime(open_trades['date'])
        open_trades['Year'] = open_trades['date'].dt.year


        #---------------------------#
        open_trades['EntryDate Year'] = open_trades['date'].dt.year
        open_trades['EntryDate Month'] = open_trades['date'].dt.month

        open_trades['ClosingDate Year'] = open_trades['Closing Date'].dt.year
        open_trades['ClosingDate Month'] = open_trades['Closing Date'].dt.month



        # ------------------- Filtering Fwd Testing Data and Backward Testing Data ---------------------- #

#         BT_data = open_trades[(open_trades['date'] < dt.datetime(2020,10,1,0,0,0))  & (open_trades['date'] >= dt.datetime(2011,1,1,0,0,0)) ]
#         FT_data = open_trades[open_trades['date'] >= dt.datetime(2020,10,1,0,0,0)]
        BT_data = open_trades





        #---calculation of ppm by trade opening date ------#
        data_pos_1 = pd.DataFrame()
        data_pos_1 = BT_data.groupby(by = ['EntryDate Year','EntryDate Month'])["Profit_Loss"].sum()
        data_pos_1 = pd.DataFrame(data_pos_1)
        data_pos_1.reset_index(inplace=True)
        data_pos_1['Positive_months by Trade Open Date'] = np.where(data_pos_1["Profit_Loss"] > 0,1,0)


        #---calculation of ppm by trade closing date ------#
        data_pos_2 = pd.DataFrame()
        data_pos_2 = BT_data.groupby(by = ['ClosingDate Year','ClosingDate Month'])["Profit_Loss"].sum()
        data_pos_2 = pd.DataFrame(data_pos_2)
        data_pos_2.reset_index(inplace=True)
        data_pos_2['Positive_months by Trade Close Date'] = np.where(data_pos_2["Profit_Loss"] > 0,1,0)


        # ----------------------- creating Summary ---------------------- #    

        # ------------------- BackTesting Data ---------------------- #  
        Back_testing_data = pd.DataFrame(BT_data.groupby(['Strategy','Currency','Condition','Timeframe','parameter_1','parameter_2','parameter_3','parameter_4',
                                                                'parameter_5','parameter_6']).agg(
                                                                Net_P_L=('Profit_Loss', 'sum'),
                                                                Total_Trades=('Profit_Loss', 'count'),
                                                                Profitable_Trades = ('Good_Bad_Flag','sum'))).reset_index()


        Back_testing_data['Accuracy'] = Back_testing_data['Profitable_Trades']/Back_testing_data['Total_Trades']
        Back_testing_data['PPT'] = Back_testing_data['Net_P_L']/Back_testing_data['Total_Trades']


        Back_testing_data['Total months by Trade Open Date'] = len(data_pos_1)
        Back_testing_data['Total positive Months by Trade Open Date'] = data_pos_1['Positive_months by Trade Open Date'].sum()
        Back_testing_data['% Positive Months by Trade Open Date'] = Back_testing_data['Total positive Months by Trade Open Date']/Back_testing_data['Total months by Trade Open Date']


        Back_testing_data['Total months by Trade Close Date'] = len(data_pos_2)
        Back_testing_data['Total positive Months by Trade Close Date'] = data_pos_2['Positive_months by Trade Close Date'].sum()
        Back_testing_data['% Positive Months by Trade Close Date'] = Back_testing_data['Total positive Months by Trade Close Date']/Back_testing_data['Total months by Trade Close Date']




        # -------------------------------------------------------------- #

        # -------------------Fwd Testing Data ---------------------- #

#         Fwd_testing_data = pd.DataFrame(FT_data.groupby(['Strategy','Currency','Condition','Timeframe','Window_lookback','Threshold_Long','RSI_period','RSI_period_exit']).agg(
#                                                             Net_P_L=('Profit_Loss', 'sum'),
#                                                             Total_Trades=('Profit_Loss', 'count'),
#                                                             Profitable_Trades = ('Good_Bad_Flag','sum'))).reset_index()


#         Fwd_testing_data['Accuracy'] = Fwd_testing_data['Profitable_Trades']/Fwd_testing_data['Total_Trades']
#         Fwd_testing_data['PPT'] = Fwd_testing_data['Net_P_L']/Fwd_testing_data['Total_Trades']



        # ---------------- Exporting Files ---------------------- #


        # --------- Open Trades ------------- #
        export_path_Open_Trades =  initial_export_path + "/Open_Trades/"
        
        pq.write_table(pyarrow.table(unclosed_trades), export_path_Open_Trades + str(sym1)+'_' + str(timeframe) + '_' + str(parameter_1) + '_'+
                                        str(parameter_2) + "_" + str(parameter_3) + "_" + 
                                        str(parameter_4) +  '_' + str(parameter_5) +  '_' + str(parameter_6) +  '_' +
                                            (condition) + '_Open.parquet', flavor='fastparquet')
        # ----------------------------------- #


        # --------- Trade level data Trades ------------- #
        export_path_trade_level_path =  initial_export_path + "/Trade Level Data/"
        
        pq.write_table(pyarrow.table(open_trades), export_path_trade_level_path + str(sym1)+'_' + str(timeframe) + '_' + str(parameter_1) + '_'+
                                        str(parameter_2) + "_" + str(parameter_3) + "_" + 
                                        str(parameter_4) +  '_' + str(parameter_5) +  '_' + str(parameter_6) +  '_' +
                                        (condition) + ".parquet", flavor='fastparquet')
        # ----------------------------------- #
        
#         # --------- Trade level data Trades ------------- #
#         export_path_trade_level_path =  initial_export_path + "/Trade Level Data/"

#         df.to_csv(export_path_trade_level_path + str(sym1)+'_' + str(timeframe) + '_' + str(Window_lookback) + '_'+
#                                    str(Threshold_Long) + "_" + str(RSI_period) + "_" + 
#                                    str(RSI_period_exit) +  '_Long_Full data.csv',index = False)
#         # ----------------------------------- #
        
        # --------- BT Summary Trades ------------- #
        export_path_BT =  initial_export_path + "/BT/"
        
        pq.write_table(pyarrow.table(Back_testing_data), export_path_BT + str(sym1)+'_' + str(timeframe) + '_' + str(parameter_1) + '_'+
                                        str(parameter_2) + "_" + str(parameter_3) + "_" + 
                                        str(parameter_4) +  '_' + str(parameter_5) +  '_' + str(parameter_6) +  '_' +
                                        (condition) + "_BT.parquet", flavor='fastparquet')
        # ----------------------------------- #


        # --------- FT Summary Trades ------------- #
#         export_path_FT =  initial_export_path + "/FT/"

#         Fwd_testing_data.to_csv(export_path_FT + str(sym1)+'_' + str(timeframe) + '_' + str(Window_lookback) + '_'+
#                                    str(Threshold_Long) + "_" + str(RSI_period) + "_" + 
#                                    str(RSI_period_exit) +  '_Long_FT.csv',index = False)
        # ----------------------------------- #

        clear_output()

        
    except Exception as e:
        
        print(e)
        
        # --------- Error Trades ------------- #
        
        
        error_file = pd.DataFrame()
        path_for_error_files = initial_export_path + "/Error/"

        
        pq.write_table(pyarrow.table(error_file), path_for_error_files + str(sym1)+'_' + str(timeframe) + '_' + str(parameter_1) + '_'+
                                    str(parameter_2) + "_" + str(parameter_3) + "_" + 
                                    str(parameter_4) +  '_' + str(parameter_5) +  '_' + str(parameter_6) +  '_' +
                                    (condition) + "_error.parquet", flavor='fastparquet')
        # ----------------------------------- #
        

In [12]:

cpu_count = 40

def parallel_task(func, iteration_list,cpus):

    with open(f'./tmp_func_Long.py', 'w') as file:
        file.write(inspect.getsource(func).replace(func.__name__, "task"))
        
    from tmp_func_Long import task

    if __name__ == '__main__':
        func = partial(task)
        pool = Pool(processes=cpus)
        res = pool.starmap(func,([i,iteration_list,price_data,initial_export_path] for i in range(len(iteration_list))))
        pool.close()
        pool.join()
        return res
    else:
        raise "Not in Jupyter Notebook"
# iterable = [i for i in range(len(iteration_list))]
outputs = parallel_task(Func,iteration_list,cpus = cpu_count)
